# Exploração e limpeza de dados
Vou procurar recriar os exemplos do livro Projetos de Ciência de Dados com Pyhthon, não exatamente igual, irei procurar fazer mudancas ou criar codigo que eu ache interessante e possa contribuir com meu aprendizado. Também irei tentar explicar os códigos quando possivel e explicar os calculos usados, quando eu souber. 

In [1]:
#Link do dataset que iremos usar 
#https://bit.ly/HIk5t3
#Nome do dataset: default_of_credit_card_clients__courseware_version_1_21_19.xls

In [2]:
#Import o modulo que usaremos
import pandas as pd

In [6]:
df = pd.read_excel('default_of_credit_card_clients__courseware_version_1_21_19.xls')

In [5]:
!pip install xlrd

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 KB 368.9 kB/s eta 0:00:00MB/s eta 0:00:010:01


In [7]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [8]:
#Pronto agora que carregamos o dataset, vamos examinar

In [10]:
df.shape

(30000, 25)

# O nosso cliente é uma empresa de cartão de credito, e temos um dataset com 30 mil clientes, nosso trabalho vai ser analisar os dados e descobir, qual clientes podem ficar inadiplentes no proximo mês, para isso vamos precisar usar algumas ferramentas de machine learning

In [11]:
df.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month'],
      dtype='object')

In [12]:
df['default payment next month']
#Como é possivel ver temos o 0 que provavelmente significar que a pessoa não ira pagar no proximos e mês e 1 que ira pagar
#Nosso trabalho vai ser ver quais caracteristicas mais afetam esse resultado final
#Mas antes vamos limpar e explorar o nosso dataset removendo outlier, dados irrelevantes e dados repetidos

0        1
1        1
2        0
3        0
4        0
        ..
29995    0
29996    0
29997    1
29998    1
29999    1
Name: default payment next month, Length: 30000, dtype: int64

# O que siginifica nossas variaveis

LIMIT_BAL = Valor do cŕedito fornecido(em novos dolares taiwnaeses) inclusive o crédito do consumidor individual e familiar

EDUCATION = Nivel de instrução ( 1 = pos-graduacao, 2 = universidade, 3 = ensino medio e 4 = outros)

MARRIAGE = Estado civil
AGE = idade
PAY_1 - PAY_6 = Registro de pagamentos passados, onde -1 é pagamento em dia, 1 é um mês de atraso, 2 são dois meses de atraso e assim por diante até 8 meses de atraso

PAY_1 = Setembro, PAY_2= Agosto e assim por diante até o mes de abril

BILL_AMT1 - BILL_AMT2= Valor da fatura( e segue o mesmo padrão que os registros de pagamentos)

In [13]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [16]:
#Agora vamos ver quantos Ids excluisvos nos temos , lembrando que como temos 30 mil linhas precisamos da mesma quantidade de Id únicos
df['ID'].nunique()

29687

In [20]:
id_count = df['ID'].value_counts()

In [44]:
id_count[id_count>=2]

(29687, 25)

In [23]:
id_count.value_counts()#Vamos agrupar por quantidade de vezes que o id aparece e contar

count
1    29374
2      313
Name: count, dtype: int64

In [24]:
#Agora vamo criar uma mascara que salva todos os id que aparecem duas vezes no nosso dataset outra maneira de removar a duplicates seria
#df_semduplicates = df.drop_duplicates('ID')
#df_semduplicates.shape

In [25]:
dupe_mask = id_count == 2
dupe_mask[0:5] #Os 5 primeiros

ID
ad23fe5c-7b09    True
1fb3e3e6-a68d    True
89f8f447-fca8    True
7c9b7473-cc2f    True
90330d02-82d9    True
Name: count, dtype: bool

In [26]:
id_count.index[0:5]

Index(['ad23fe5c-7b09', '1fb3e3e6-a68d', '89f8f447-fca8', '7c9b7473-cc2f',
       '90330d02-82d9'],
      dtype='object', name='ID')

In [27]:
#Nova variavel para salvar os id repetidos

dups_id = id_count.index[dupe_mask]

In [29]:
#Transformando essa nova variavel em uma list

dups_id = list(dups_id)
len(dups_id)

313

In [30]:
#Localizandos todos os ids repetidos e puxandos todas as colunas, apenas os tres primeiros id duplicados
df.loc[df['ID'].isin(dups_id[0:3]),:]

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
5033,89f8f447-fca8,320000,2,2,1,32,0,0,0,0,...,169371,172868,150827,8000,8000,5500,6100,6000,5000,0
5133,89f8f447-fca8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16727,1fb3e3e6-a68d,80000,1,2,2,33,2,2,0,0,...,27394,29922,31879,0,2000,2000,3000,2600,0,1
16827,1fb3e3e6-a68d,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29685,ad23fe5c-7b09,50000,1,3,1,32,0,0,0,0,...,12882,8131,3983,3000,2871,1000,163,3983,3771,1
29785,ad23fe5c-7b09,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
#A primeia apareçao do ID temos o valores normais e na segunda temos apenas Zero, vamos criar um maskara com o valores 0

In [32]:
df_zero_mask = df ==0

In [34]:
feature_zero_mask = df_zero_mask.iloc[:,1:].all(axis = 1)

In [35]:
sum(feature_zero_mask)

315

In [37]:
#agora vamos criar um novo dataframe excluindo todos a linhas que tem apenas zero com execao do index

In [38]:
df_clean_1 = df.loc[~feature_zero_mask,:].copy() # Vamos criar uma copia idependente

In [39]:
df_clean_1.shape

(29685, 25)

In [40]:
#Pronto removemos todos os valores repetidos do nosso dataset

In [41]:
df_clean_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29685 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ID                          29685 non-null  object
 1   LIMIT_BAL                   29685 non-null  int64 
 2   SEX                         29685 non-null  int64 
 3   EDUCATION                   29685 non-null  int64 
 4   MARRIAGE                    29685 non-null  int64 
 5   AGE                         29685 non-null  int64 
 6   PAY_1                       29685 non-null  object
 7   PAY_2                       29685 non-null  int64 
 8   PAY_3                       29685 non-null  int64 
 9   PAY_4                       29685 non-null  int64 
 10  PAY_5                       29685 non-null  int64 
 11  PAY_6                       29685 non-null  int64 
 12  BILL_AMT1                   29685 non-null  int64 
 13  BILL_AMT2                   29685 non-null  int64 
